In [ ]:
!pip install pennylane

In [ ]:
import os
import json
import pennylane as qml
import torch
from google.colab import drive
from datetime import datetime
import pennylane.numpy as np
import autograd.numpy as anp
import matplotlib.pyplot as plt
from keras.datasets import mnist
from skimage.transform import resize
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from typing import List, Tuple, Optional
import tensorflow as tf
from collections import Counter
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

device = 'cuda' if torch.cuda.is_available() else 'cpu'

np.random.seed(42)

In [ ]:
print(f"device being used --- {device}")

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Define the file path within Google Drive
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
file_path = f'/content/drive/My Drive/Result/results_{current_time}.json'
model_params_file_path = f'/content/drive/My Drive/Result/results_params_{current_time}.json'

# Fetch Data
1. choosing small datasize initally to experiment with the model.
2. shuffling then and then making train, validation and test splits

In [ ]:
# datasize to choose for training, validation and test set
train_datasize = 2000
test_datasize = 1000
# fetch data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# shuffle the training data
train_indices = np.random.permutation(len(x_train))
x_train = x_train[train_indices]
y_train = y_train[train_indices]

# shuffle the test data
test_indices = np.random.permutation(len(x_test))
x_test = x_test[test_indices]
y_test = y_test[test_indices]

# slice the datasize
x_train = x_train[:train_datasize]
y_train = y_train[:train_datasize]
x_test = x_test[:test_datasize]
y_test = y_test[:test_datasize]

# count the number of each class in x_train
train_class_counts = Counter(y_train)

# the class counts
print("Class counts in x_train:")
for label, count in train_class_counts.items():
    print(f"Class {label}: {count}")

def check_imbalance(class_counts, datasize):
  avg_count = datasize / 10
  # taking imbalance threshold, 20% of average count
  threshold = 0.2 * avg_count
  for label, count in class_counts.items():
    if abs(count - avg_count) > threshold:
      return True, label, count
  return False, None, None

# check for imbalance in training data
is_imbalanced_train, train_imbalanced_class, train_imbalanced_count = check_imbalance(train_class_counts, train_datasize)
if is_imbalanced_train:
    print(f"\nImbalance detected in training data for class {train_imbalanced_class} with count {train_imbalanced_count}")
else:
    print("\nNo significant imbalance detected in training data")
# split the training data into training and test sets
# X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1667, random_state=42)
# print(f"Data for model --- training: {X_train.shape[0]} validation: {X_val.shape[0]} test: {X_test.shape[0]}")

In [ ]:
# plot some to show
fig = px.imshow(x_train[:10, :, :], binary_string=True, facet_col=0, facet_col_wrap=5)
fig.show()

# normalize the images data
X_train, X_test = x_train[..., np.newaxis] / 255.0, x_test[..., np.newaxis] / 255.0
Y_train = y_train
Y_test = y_test

# Data Reduction

In [ ]:
X_train = tf.image.resize(X_train[:], (256, 1)).numpy()
X_test = tf.image.resize(X_test[:], (256, 1)).numpy()
X_train, X_test = tf.squeeze(X_train).numpy(), tf.squeeze(X_test).numpy()

In [ ]:
# setup params for circuit training
U_params = 15
total_params = U_params * 3 + 2 * 3
n_wires = 8
dev = qml.device("default.qubit", wires=n_wires)

# randomly initialize the parameters using numpy, we can try later using xavier uniform
params = np.random.randn(total_params, requires_grad=True)

# Quantum ciruit to be used for convolution
def U_SU4(params, wires):  # 15 params
    qml.U3(params[0], params[1], params[2], wires=wires[0])
    qml.U3(params[3], params[4], params[5], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.RY(params[6], wires=wires[0])
    qml.RZ(params[7], wires=wires[1])
    qml.CNOT(wires=[wires[1], wires[0]])
    qml.RY(params[8], wires=wires[0])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.U3(params[9], params[10], params[11], wires=wires[0])
    qml.U3(params[12], params[13], params[14], wires=wires[1])

# Quantum Circuits for Convolutional layers
def conv_layer1(U, params):
    U(params, wires=[0, 7])
    for i in range(0, 8, 2):
        U(params, wires=[i, i + 1])
    for i in range(1, 7, 2):
        U(params, wires=[i, i + 1])

def conv_layer2(U, params):
    U(params, wires=[0, 6])
    U(params, wires=[0, 2])
    U(params, wires=[4, 6])
    U(params, wires=[2, 4])

def conv_layer3(U, params):
    U(params, wires=[0, 4])
    U(params, wires=[4, 2])

# Quantum Circuits for Pooling layers
def pooling_layer1(V, params):
    for i in range(0, 8, 2):
        V(params, wires=[i + 1, i])

def pooling_layer2(V, params):
    V(params, wires=[2, 0])
    V(params, wires=[6, 4])

def Pooling_ansatz(params, wires):  # 2 params
    qml.CRZ(params[0], wires=[wires[0], wires[1]])
    qml.PauliX(wires=wires[0])
    qml.CRX(params[1], wires=[wires[0], wires[1]])

# define circuit layers
def QCNN_structure_modified(U, params, U_params):
    param1 = params[0:U_params]  # 15 params
    param2 = params[U_params:2 * U_params]  # 15 params
    param3 = params[2 * U_params:3 * U_params]  # 15 params
    param4 = params[3 * U_params:3 * U_params + 2]  # 2 params
    param5 = params[3 * U_params + 2:3 * U_params + 4]  # 2 params
    #param6 = params[3 * U_params + 4:3 * U_params + 6]  # 2 params

    # print(f"total params -- {params.shape} U_params {U_params}")
    # print(
    #     f"Params: {param1.shape} {param2.shape} {param3.shape} {param4.shape} {param5.shape}"
    # )
    # layer 1
    conv_layer1(U, param1)
    pooling_layer1(Pooling_ansatz, param4)
    # layer 2
    conv_layer2(U, param2)
    pooling_layer2(Pooling_ansatz, param5)
    # layer 3
    conv_layer3(U, param3)

# define circuit
@qml.qnode(dev)
def QCNN(X, params, U_params, embedding_type="Amplitude", cost_fn="cross_entropy"):
  # data encoding
  qml.AmplitudeEmbedding(X, wires=range(8), normalize=True)
  # circuit with params
  QCNN_structure_modified(U_SU4, params, U_params=U_params)

  # compute cost_fun
  result_16_states = qml.probs(wires=[0, 1, 2, 3])
  return result_16_states

# draw the circuit
x_sample = X_test[0].reshape(1, X_test[0].shape[0])
print(qml.draw_mpl(QCNN)(x_sample, params, U_params))

In [ ]:
# define loss and accuracy functions
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def multi_class_cross_entropy(labels: List, predictions: List, num_classes: Optional[int] = 10):
    """
    Compute the cross-entropy loss between ground truth labels and predicted probabilities.

    Args:
    - labels (array): Ground truth labels, shape (num_samples,)
    - predictions (array): Predicted probabilities for each class, shape (num_samples, num_classes)

    Returns:
    - loss (float): Cross-entropy loss
    """
    num_samples = len(labels)
    loss = 0
    for i in range(num_samples):
        label = labels[i]
        prediction = predictions[i]
        # FIXME: as the probabilites from qml are not in between 0 and 1, we need to normalize them apply softmax function
        softmax_probabilites = softmax(prediction)
        # testing the predicted class label
        # predicted_class_label = np.argmax(softmax_probabilites)
        c_entropy = -anp.log(softmax_probabilites[label])
        loss += c_entropy
    return loss / num_samples

def cost(params, X, Y, U_params, embedding_type="Amplitude", cost_fn="cross_entropy"):
    # we need predictions for 10 classes only
    predictions = [QCNN(x, params, U_params, embedding_type=embedding_type, cost_fn=cost_fn)[:10] for x in X]
    loss = multi_class_cross_entropy(Y, predictions)
    return loss

def get_predicted_labels_QCNN(params, X, Y, U_params, embedding_type="Amplitude", cost_fn="cross_entropy"):
    # we need predictions for 10 classes only
    predictions = [QCNN(x, params, U_params, embedding_type=embedding_type, cost_fn=cost_fn)[:10] for x in X]
    softmax_predictions = [softmax(p) for p in predictions]
    # get predicted labels
    predicted_labels = np.argmax(softmax_predictions, axis=1)
    return predicted_labels

def accuracy_test(predictions, labels):
  acc = 0
  for label, pred in zip(labels, predictions):
      if np.argmax(pred) == label:
          acc = acc + 1
  return acc / len(labels)

In [ ]:
# Hyperparamters
n_epochs = 30
batch_size = 64
initial_lr = 0.01
patience = 2
lr_factor = 0.1
min_lr = 1e-6

opt = qml.NesterovMomentumOptimizer(stepsize=initial_lr)

tr_steps_per_epoch = len(X_train) // batch_size
#val_steps_per_epoch = len(X_val) // batch_size

train_loss_history = []
train_acc_history = []
val_loss_history = []
val_acc_history = []

best_tra_loss = float('inf')
best_train_acc = 0
epochs_no_improve = 0
# track the current learning rate
current_lr = initial_lr

for epoch in range(n_epochs):
  total_samples = 0
  total_loss = 0
  correct_count = 0
  # shuffle the data for each epoch
  indices = np.random.permutation(len(X_train))
  X_train_shuffled = X_train[indices]
  Y_train_shuffled = Y_train[indices]
  for step in range(tr_steps_per_epoch):
      # create mini-batch
      X_batch = X_train_shuffled[step * batch_size: (step + 1) * batch_size]
      Y_batch = Y_train_shuffled[step * batch_size: (step + 1) * batch_size]
      prev_params = params
      params, cost_new = opt.step_and_cost(
            lambda v: cost(v, X_batch, Y_batch, U_params,),
            params,
        )
      predicted_labels = get_predicted_labels_QCNN(prev_params, X_batch, Y_batch, U_params=U_params)
      correct_count += np.sum(predicted_labels == Y_batch)
      total_samples += len(Y_batch)
      total_loss += cost_new * len(Y_batch)
  # average accuracy for the epoch
  train_accuracy = correct_count / total_samples
  # average loss for the epoch
  train_average_loss = total_loss / total_samples
  train_loss_history.append(train_average_loss)
  if isinstance(train_accuracy, qml.numpy.tensor):
      train_accuracy = train_accuracy.item()
  train_acc_history.append(train_accuracy)
  # log training details
  print(f"Epoch {epoch + 1}/{n_epochs}, Average Loss: {train_average_loss:.4f}")
  print(f"Epoch {epoch + 1}/{n_epochs}, Accuracy: {train_accuracy:.4f}")

  # Check for improvement in training accuracy
  if train_accuracy > best_train_acc:
    best_train_acc = train_accuracy
    epochs_no_improve = 0
  else:
    epochs_no_improve += 1

  # Reduce learning rate if no improvement for 'patience' epochs
  if epochs_no_improve >= patience:
    current_lr = max(current_lr * lr_factor, min_lr)  # Ensure learning rate does not go below min_lr
    opt = qml.NesterovMomentumOptimizer(stepsize=current_lr)  # Update optimizer with the new learning rate
    epochs_no_improve = 0  # Reset patience counter
    print(f"Reducing learning rate to {current_lr}")

  # validation phase
  # val_total_samples = 0
  # val_total_loss = 0
  # val_correct_count = 0
  # for step in range(val_steps_per_epoch):
  #     # Create mini-batch
  #     X_val_batch = X_val[step * batch_size: (step + 1) * batch_size]
  #     Y_val_batch = Y_val[step * batch_size: (step + 1) * batch_size]
  #     val_cost_new = cost(params, X_val_batch, Y_val_batch, U_params)
  #     val_predicted_labels = get_predicted_labels_QCNN(params, X_val_batch, Y_val_batch, U_params=U_params)
  #     val_correct_count += np.sum(val_predicted_labels == Y_val_batch)
  #     val_total_samples += len(Y_val_batch)
  #     val_total_loss += val_cost_new * len(Y_val_batch)
  # val_accuracy = val_correct_count / val_total_samples
  # val_average_loss = val_total_loss / val_total_samples
  # if isinstance(val_average_loss, qml.numpy.tensor):
  #     val_average_loss = val_average_loss.item()
  # if isinstance(val_accuracy, qml.numpy.tensor):
  #     val_accuracy = val_accuracy.item()
  # val_loss_history.append(val_average_loss)
  # val_acc_history.append(val_accuracy)
  # # log validation details
  # print(f"Epoch {epoch + 1}/{n_epochs}, Validation Loss: {val_average_loss:.4f}")
  # print(f"Epoch {epoch + 1}/{n_epochs}, Validation Accuracy: {val_accuracy:.4f}")
  # lets try doing after every 10 epochs
  # if (epoch + 1) % 10 == 0:
  #   # reduce the current learning rate
  #   current_lr *= lr_factor
  #   opt = qml.NesterovMomentumOptimizer(stepsize=current_lr)
  #   print(f"Reducing learning rate to {current_lr}")

  # Learning rate scheduling
  # if val_average_loss < best_val_loss:
  #     best_val_loss = val_average_loss
  #     epochs_no_improve = 0
  # else:
  #     epochs_no_improve += 1

  # if epochs_no_improve >= patience:
  #   new_lr = max(current_lr * lr_factor, min_lr)
  #   if new_lr < current_lr:
  #       print(f"Reducing learning rate from {current_lr} to {new_lr}")
  #       current_lr = new_lr
  #       opt = qml.AdamOptimizer(stepsize=current_lr)
  #   epochs_no_improve = 0

In [ ]:
params_data = params.tolist()

In [ ]:
# save model params to file
if not os.path.exists(os.path.dirname(model_params_file_path)):
  os.makedirs(os.path.dirname(model_params_file_path))
with open(model_params_file_path, 'w') as loss_f:
    json.dump(params_data, loss_f, indent=4)
print(f"model params saved to {model_params_file_path}")

In [ ]:
# load the paramsfile or data file if you want to plot the loss and acc plots
training_data_file_path = '/content/drive/My Drive/Result/results_params_20240603_222522.json'
with open(training_data_file_path, 'r') as f:
  loaded_params = json.load(f)

In [ ]:
# run on test set
test_predictions = [QCNN(x, loaded_params, U_params)[:10] for x in X_test[:2]]
# predictions are to be converted to softmax probabilities
softmax_predictions = [softmax(p) for p in test_predictions]
test_accuracy = accuracy_test(softmax_predictions, Y_test)
print(f"Test Accuracy {test_accuracy:.4f}")

In [ ]:
def remove_nan_tensors(tensor_list):
  clean_tensors = []
  nan_indices = []
  for i, tensor in enumerate(tensor_list):
    if np.isnan(tensor).any().item():
      nan_indices.append(i)
    else:
      clean_tensors.append(tensor)
  return clean_tensors, nan_indices

# clean the list of tensors and get indices of tensors with NaNs
clean_tensors, nan_indices = remove_nan_tensors(softmax_predictions)
if nan_indices:
  print(f"found some indices to be nan {len(nan_indices)}")
  # remove those indexs from the list of test image and its label
  for index in sorted(nan_indices, reverse=True):
    softmax_predictions.pop(index)
    Y_test.pop(index)

In [ ]:
# Calculate the confusion matrix
predicted_test_labels = np.array([np.argmax(pred) for pred in softmax_predictions])
cm = confusion_matrix(Y_test[:2], predicted_test_labels)
all_labels = np.unique(np.concatenate((Y_test[:2], predicted_test_labels)))
# Plot the confusion matrix using sklearn's ConfusionMatrixDisplay
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=all_labels)
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
# save results to drive
def convert_to_float(lst):
    return [float(item) for item in lst]

data = {
    "train_loss": convert_to_float(train_loss_history),
    "train_acc": convert_to_float(train_acc_history),
    "val_loss": convert_to_float(val_loss_history),
    "val_acc": convert_to_float(val_acc_history),
}

# Check if the directory exists, if not, create it
if not os.path.exists(os.path.dirname(file_path)):
  os.makedirs(os.path.dirname(file_path))
with open(file_path, 'w') as loss_f:
    json.dump(data, loss_f, indent=4)
print(f"Data saved to {file_path}")

In [ ]:
def plot_QCNN_metric(data, title, y_axis_title, name):
    # Create the plot
    fig = go.Figure()
    # Add the data trace
    fig.add_trace(go.Scatter(y=data, mode='lines', name=name))
    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title='Epoch',
        yaxis_title=y_axis_title,
        legend_title='Legend'
    )
    fig.show()

In [ ]:
plot_QCNN_metric(data=train_loss_history, title='Training Loss Over Epochs', y_axis_title='Loss', name='Training Loss')

In [ ]:
plot_QCNN_metric(data=train_acc_history, title='Training Acc Over Epochs', y_axis_title='Acc', name='Training Acc')